<a href="https://colab.research.google.com/github/BingHungLee/EDC_Cracking_Furnace_Design/blob/main/%5B20210712%5D_EDC_Fireheater_Cantera.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
################################################################################
# INSTALL CONDA AND CANTERA ON GOOGLE COLAB
################################################################################
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

--2021-07-12 03:26:39--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85055499 (81M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh.1’

Miniconda3-py37_4.8 100%[===================>]  81.12M   174MB/s    in 0.5s    

2021-07-12 03:26:40 (174 MB/s) - ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh.1’ saved [85055499/85055499]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: / - \ done

# All requested packages already installed.

installation finished.
    You currently have a PYTHONPATH environment variable set. This may cause
    unexpected behavior when running the Python interpreter in Miniconda3.
    For best results, please verify that your PYTHONPATH only points to
    directories

In [3]:
!conda install --channel cantera cantera ipython matplotlib jupyter

Solving environment: \ | / - \ | / - \ | / - \ | / - \ | / - \ | failed with initial frozen solve. Retrying with flexible solve.
Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ done

## Package Plan ##



In [4]:
!pip install silence_tensorflow
!pip install sklearn
!pip install keras
!pip install tensorflow

  Created wheel for silence-tensorflow: filename=silence_tensorflow-1.1.1-py3-none-any.whl size=3737 sha256=ae44da43919ab39f7ab726500057ed8afc510e6508ea4cf6eef662995b14620c
  Stored in directory: /root/.cache/pip/wheels/a2/42/cf/26148cd545b324903a62edde56cfc4502b278c104a07bb6b96
Successfully built silence-tensorflow
     |████████████████████████████████| 22.3 MB 1.3 MB/s 
     |████████████████████████████████| 28.5 MB 28 kB/s 
     |████████████████████████████████| 303 kB 47.6 MB/s 
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1315 sha256=d2088af76902f48daa941673b38e9bf8080b0cfccd334c614a11782d2a49fbf9
  Stored in directory: /root/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built sklearn
     |████████████████████████████████| 636 kB 22.9 MB/s 
     |████████████████████████████████| 454.3 MB 12 kB/s 
     |████████████████████████████████| 6.0 MB 23.2 MB/s 
     |████████████████████████████████| 42 kB 74

In [7]:
import cantera as ct
import numpy as np
from os.path import dirname, join
from os import system
import sys
from cantera import add_directory
from pickle import load
from keras.layers import Input, Dense, Activation, concatenate
from keras.models import Model
from keras import losses
from keras import optimizers
from argparse import ArgumentParser
from numpy import zeros, arange, zeros_like, sum, hstack
from cantera import Species, Solution, IdealGasReactor, MassFlowController, Reservoir, SolutionArray, MassFlowController, PressureController, ReactorNet
from silence_tensorflow import silence_tensorflow

In [10]:
reaction_mech = 'chem_annotated_irreversible_aspen.xml'

In [15]:
def EDC_cracking(
        reaction_mech,
        lst,
        p,
        CCl4,
        mfr,
        n_steps=1000,
        n_pfr=18,
        l=18,
        A=0.03225097679
        ):
  lst = [i + 273.15 for i in lst]
  p = p * 98066.5 + 101325
  spcs = Species.listFromFile(reaction_mech)
  for spc in spcs[::-1]:
    if spc.composition == {'C': 2.0, 'Cl': 2.0, 'H': 4.0} and spc.charge == 0:
      EDC_l = spc.name
    if spc.composition == {'C': 1.0, 'Cl': 4.0} and spc.charge == 0:
      CCl4_l = spc.name
  EDC = 1 - CCl4
  __compos__ = '{}:{}, {}:{}'.format(EDC_l, EDC, CCl4_l, CCl4)
  model = Solution(reaction_mech)
  model.TPY = lst[0], p, __compos__
  r = IdealGasReactor(model)
  r.volume = A * l / n_steps
  up = Reservoir(model, name='up')
  down = Reservoir(model, name='down')
  m = MassFlowController(up, r, mdot=mfr)
  v = PressureController(r, down, master=m, K=1e-5)
  sim = ReactorNet([r])
  z = (arange(n_steps) + 1) * l / n_steps
  __t__ = zeros(n_pfr)
  __compos__ = [None] * n_pfr
  __cracks__ = [0]
  states = SolutionArray(r.thermo)
  for i, T in enumerate(lst[1:]):
    dT = (lst[i+1] - lst[i]) / n_steps
    t_r = zeros_like(z)
    for n in range(n_steps):
      model.TP = lst[i] + (n + 1) * dT, None
      r.syncState()
      model.TPY = r.thermo.TPY
      up.syncState()
      sim.reinitialize()
      sim.set_initial_time(0)
      sim.advance_to_steady_state()
      t_r[n] = r.mass / mfr
      states.append(r.thermo.state)
    __t__[i] = sum(t_r)
    __compos__[i] = model.Y[4:]
    Y = (EDC - model.Y[model.species_index(EDC_l)]) / EDC
    __cracks__.append(Y)
  return __compos__, __t__, __cracks__



In [37]:
mfr = 53.
p = 13.
CCl4 = 1000.
#lst = [324.7,350,374.7,399.3,424,451.3,460.7,466,471.3,476.7,478.7,479.3,480,481.3,482.3,483.3,484.3,485.3,486.7]
lst = [350,368,382,400,409,415,421,427,433,440,445,450,453,456,459,462,465,468,471]

#y_pre = [0]
#n_steps = 100
n_pfr = len(lst)-1
l = 18
mfr /= 3.6
CCl4 /= 1e6
A = 0.0321263 if n_pfr == 18 else 0.05388554

compos, t, cracks = EDC_cracking(reaction_mech,lst,p,CCl4,mfr,n_steps=1000,n_pfr=18,l=18,A=0.03225097679)

In [38]:
compos

[array([9.98998741e-01, 5.53560856e-15, 4.27540759e-19, 1.00000000e-03,
        7.46935971e-19, 4.63730679e-07, 8.33985097e-14, 7.94897740e-07,
        4.73185801e-21, 3.25058293e-14, 1.34323607e-17, 6.12765617e-14,
        3.55877863e-28, 2.59613695e-16, 2.17973462e-40, 8.51937721e-27,
        2.31809186e-16, 3.49063790e-22, 1.40642277e-17, 3.62751806e-18,
        2.60479457e-19, 1.53992262e-33, 4.09163442e-30, 2.93051103e-29,
        0.00000000e+00, 7.33920662e-29, 2.76418329e-28]),
 array([9.98985655e-01, 4.14485953e-14, 1.51290505e-18, 1.00000000e-03,
        4.74614837e-17, 5.28544075e-06, 4.55745071e-13, 9.05996767e-06,
        4.32445632e-19, 1.88127505e-13, 9.54829353e-16, 7.73147170e-12,
        2.53537685e-25, 3.14709150e-14, 1.58787461e-36, 1.03002721e-23,
        2.81535011e-14, 6.06386080e-19, 1.82545382e-14, 7.16994364e-16,
        3.58074185e-17, 2.21282463e-29, 1.17129919e-25, 7.34533222e-26,
        0.00000000e+00, 5.01786230e-25, 7.94885970e-25]),
 array([9.98900772e-

In [39]:
print("t:",t)
print("sum(t):",sum(t))

t: [1.02211525 0.99685306 0.97281984 0.95319247 0.94204719 0.93231571
 0.92152189 0.90915806 0.89386724 0.87588728 0.85513083 0.83161441
 0.80652625 0.78037199 0.75451084 0.72991368 0.70716371 0.68648098]
sum(t): 15.571490666306774


In [40]:
cracks

[0,
 1.2598884903871256e-06,
 1.4359776652379704e-05,
 9.932723134310807e-05,
 0.000516802971711266,
 0.0016649329159879134,
 0.003926360799364468,
 0.007804635059838695,
 0.014054365914742642,
 0.023852953576504927,
 0.03871834152739121,
 0.05951548988107206,
 0.08620868963558491,
 0.11726031539232834,
 0.1510215806436218,
 0.1859511237825689,
 0.22071504488849022,
 0.254320864872538,
 0.2862303063852004]